In [26]:
import pandas as pd
import numpy as np


In [39]:
report = pd.read_csv('../report.csv')
report = report.sort_values('pb')
report = report[report.pb<0.4238]
report

,cv,file,ho,pb,train_mean
25,0.415431,oldModel_RNNModelTag_ebdNANExclude_TagNAExclud...,0.000000,0.423578,0.410798
32,0.415551,oldModel_RNNModelTag_ebdNANExclude_Tag30NAExcl...,0.000000,0.423652,0.410961
0,0.414837,new_model_rnnEBD,0.424599,0.423665,NaN
20,0.415106,oldModel_RNNModelTag_containNAN,0.424900,0.423687,0.410039
26,0.415427,oldModel_RNNModelTag_ebdNANExclude_Tag75NAExcl...,0.000000,0.423695,0.410851
17,0.414803,V1Model_RNNModelTag,0.424531,0.423725,0.409778
11,0.415046,oldModel_RNNModelTag_Fold5,0.424899,0.423736,0.410336
23,0.415512,V1Model_RNNModelTag_ebdNANExclude_noholdout,0.000000,0.423750,0.409914
18,0.415270,oldModel_RNNModelTag_adversarial_ho,0.415078,0.423757,0.410434
9,0.414804,oldModel_Utag_RNN,0.424250,0.423760,0.409515


# new_model_rnnEBD 6 oldModel_Utag_RNN 4 PB 0.423628

In [28]:
files = report.file.values[:10]


In [40]:
files = report.file.values[:10]
weights = [3,2,2,2,1,1,1,1,1,1]
save = pd.DataFrame()
sigma_list=[]
weight_sum = 0
for each,w in zip(files,weights):
    df_read = pd.read_csv('../test/{}.csv'.format(each))
    if 'instance_id' not in save.columns:
        save['instance_id'] = df_read['instance_id'].values
    sigma_list.append(list(df_read['predicted_score'].values*w))
    weight_sum+=w
sigma = np.array(sigma_list)
sigma = np.sum(sigma,axis=0) / weight_sum
save['predicted_score'] = sigma

In [41]:
save.to_csv('bend_10.csv',index=False)

# oof generator

In [29]:
tr=pd.DataFrame()
te = pd.DataFrame()
for f in report.file.values[:8]:
    re_tr = pd.read_pickle('../train/{}.pkl'.format(f))
    re_te = pd.read_csv('../test/{}.csv'.format(f))
    if 'instance_id' not in tr.columns:        
        tr['instance_id'] = re_tr['instance_id'].values
        te['instance_id'] = re_te['instance_id'].values
    tr[f] = re_tr['predicted_score'].values
    te[f] = re_te['predicted_score'].values

In [30]:
print(tr.shape)
print(te.shape)

(1001650, 9)
(40024, 9)


In [31]:
tr.to_csv('../train_oof_1006.csv',index=False)
te.to_csv('../test_oof_1006.csv',index=False)

# View Best

In [33]:
import pickle

train_ori = pd.read_pickle('../../original/train.pkl')
train_best = pd.read_pickle('../train/new_model_rnnEBD.pkl')
train_index = pickle.load(open('../../original/train_index.pkl','rb'))
holdout_index = pickle.load(open('../../original/holdout_index.pkl','rb'))

In [34]:
view = pd.DataFrame()
view['predicted_score'] = train_best.loc[holdout_index,'predicted_score'].values
view['click'] = train_ori.loc[holdout_index,'click'].values

In [38]:
view[view['predicted_score']>0.7]

,predicted_score,click
69,0.708072,0
81,0.701861,1
87,0.758962,1
98,0.723295,1
145,0.778169,1
229,0.709425,1
230,0.726098,1
303,0.791796,1
436,0.774881,1
457,0.711167,0
